In [55]:
import matplotlib.pyplot as plt
from sklearn.metrics import PrecisionRecallDisplay
import numpy as np
import json
import requests
import pandas as pd

In [70]:
df = pd.read_csv('../data/data.csv')

relevant_ids = ['tt1430607', 'tt13381368', 'tt0104940',  'tt13320784', 'tt13356246', 'tt6269192', 'tt0319343', 'tt0314331', 'tt0107688', 'tt3626180',  'tt1519929', 'tt1327788', 'tt2990140', 'tt1182345', 'tt1446714', 'tt1454468', 'tt0448134', 'tt0816692', 'tt0119116', 'tt0119177', 'tt0118884', 'tt0796366', 'tt1408101', 'tt0134847', 'tt2015381', 'tt3659388', 'tt3896198', 'tt0379786', 'tt3748528','tt21157976', 'tt15609374', 'tt12483148', 'tt5597272', 'tt14191506', 'tt7794734', 'tt11771702', 'tt9782756', 'tt13347662', 'tt4372340', 'tt7918178', 'tt3351386', 'tt6024606', 'tt4859370', 'tt12787166', 'tt15663664', 'tt0435679', 'tt2582846', 'tt2582496', 'tt2962876', 'tt1714206', 'tt0910970', 'tt1282140', 'tt0377092', 'tt2205697', 'tt0398286', 'tt0109830', 'tt1798709', 'tt0467406', 'tt2980648', 'tt1659337', 'tt1010048', 'tt0817177', 'tt0181875', 'tt0338013', 'tt0338013', 'tt1924396', 'tt0414387', 'tt0332280',  'tt1859438', 'tt0462610', 'tt1838737', 'tt3846674', 'tt6472976', 'tt1638002', 'tt2674426', 'tt0147800', 'tt1022603', 'tt3783958', 'tt2194499']

filtered_df = df[~df['tconst'].isin(relevant_ids)]
relevant_sample = df[df['tconst'].isin(relevant_ids)]


In [71]:
sample = filtered_df.sample(n=222)
sample.append(relevant_sample)


/var/folders/04/05jdn06x4bd37twbh8c046nr0000gn/T/ipykernel_15220/1593244959.py:2: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  sample.append(relevant_sample)


tconst                       primaryTitle  \
39283  tt2880472                     Out of Nothing   
34835  tt2146166  Clash of Colors: LA Riots of 1992   
49604  tt5649148                     Sebastian Says   
52684  tt6593698              Death in the Terminal   
42019  tt3528486                 The Irish Prisoner   
...          ...                                ...   
51738  tt6269192          Journey Back to Christmas   
52366  tt6472976                    Five Feet Apart   
55820  tt7794734                      American Moon   
56050  tt7918178                            Spheres   
59581  tt9782756       Apollo: Missions to the Moon   

                           originalTitle  isAdult  startYear runtimeMinutes  \
39283                     Out of Nothing        0     2014.0             90   
34835  Clash of Colors: LA Riots of 1992        0     2012.0             81   
49604                     Sebastian Says        0     2016.0             \N   
52684              Death in the Terminal        0     2016.0             52   
42019             El Prisionero Irlandés        0     2015.0            100   
...                                  ...      ...        ...            ...   
51738          Journey Back to Christmas        0     2016.0             84   
52366                    Five Feet Apart        0     2019.0            116   
55820                      American Moon        0     2017.0            214   
56050                            Spheres        0     2018.0             45   
59581       Apollo: Missions to the Moon        0     2019.0             94   

                             genres  averageRating  numVotes  \
39283  Biography,Documentary,Family            7.8      99.0   
34835                   Documentary            7.1      28.0   
49604                        Comedy            9.0      44.0   
52684                   Documentary            7.6     105.0   
42019         Drama,History,Romance            7.3      34.0   
...                             ...            ...       ...   
51738          Drama,Family,Fantasy            7.0    2781.0   
52366                 Drama,Romance            7.2   66415.0   
55820           Documentary,History            8.2     200.0   
56050            Documentary,Sci-Fi            7.2     111.0   
59581           Documentary,History            7.5     528.0   

                                         directors  \
39283                                  Chad DeRosa   
34835                                 David D. Kim   
49604                                  Scott Ellis   
52684        Tali Shemesh, Asaf Sudri, Dorit Inbar   
42019  Marcela Silva y Nasute, Carlos Jaureguialzo   
...                                            ...   
51738                                   Mel Damski   
52366                               Justin Baldoni   
55820                             Massimo Mazzucco   
56050                                 Eliza McNitt   
59581                                 Tom Jennings   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                      

In [72]:
# sample.to_csv('../data/sample.csv', index=False)

In [74]:

sample[0:74].to_csv('../data/bia.csv', index=False)
sample[74:148].to_csv('../data/joni.csv', index=False)
sample[148:222].to_csv('../data/maggy.csv', index=False)


In [75]:
pd.set_option('display.max_rows', 150)
pd.set_option('display.max_colwidth', None)


In [ ]:

df = pd.read_csv('../data/bia.csv')
df[['tconst', 'originalTitle', 'genres', 'synopsis']]


In [ ]:
CHRISTMAS_QRELS_FILE = '../qrels_files/christmas_movies.txt'
TWENTIES_QRELS_FILE = '../qrels_files/20s_movies.txt'
ROMANCE_TEEN_QRELS_FILE = '../qrels_files/romance_teen.txt'
SPACE_QRELS_FILE = '../qrels_files/space_movies.txt'

CHRISTMAS_QUERY_URL = 'http://localhost:8983/solr/imdb_movies/select?bq=primaryTitle%3A%20christmas%5E5.0%20synopsis%3Achristmas%5E3&defType=dismax&fl=score%20*&indent=true&q.op=OR&q=christmas%5E2%20santa%5E2%20snow%20elf%20rodolf%20festive%20claus%20merry%20holliday%20%22candy%20cane%22%20%22christmas%20tree%22%20eve%20advent&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1'
TWENTIES_QUERY_URL = 'http://localhost:8983/solr/imdb_movies/select?bq=primaryTitle%3A%20christmas%5E5.0%20synopsis%3Achristmas%5E3&defType=dismax&fl=score%20*&indent=true&q.op=OR&q=christmas%5E2%20santa%5E2%20snow%20elf%20rodolf%20festive%20claus%20merry%20holliday%20%22candy%20cane%22%20%22christmas%20tree%22%20eve%20advent&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1'
ROMANCE_TEEN_QUERY_URL = 'http://localhost:8983/solr/imdb_movies/select?bq=genres%3Adrama%20genres%3Aromance&defType=dismax&fl=score%20*&indent=true&q.op=OR&q=romance%20teen%20crush%20heart-break%20%5C%22in%20love%5C%22%20high-school%20college%20friends%20friendship%20campus%20gossip%20passion%20attraction&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1'
SPACE_QUERY_URL = 'http://localhost:8983/solr/imdb_movies/select?bq=genres%3Asci-fi%5E3.0%20synopsis%3Astars&defType=dismax&fl=score%20*&indent=true&q.op=OR&q=space%20astronaut%20galaxy%20planets&qf=originalTitle%5E1.5%20primaryTitle%5E1.5%20synopsis%5E2&tie=0.1'

In [ ]:
# Read qrels to extract relevant documents
relevant = list(map(lambda el: el.strip(), open(CHRISTMAS_QRELS_FILE).readlines()))
# Get query results from Solr instance
results = requests.get(CHRISTMAS_QUERY_URL).json()['response']['docs']
